## Single Model Approach
This approach uses a single Consumption model to compute the prediction.
Taking as input Latitude, Longitude, Month and Lane Count

In [1]:
import numpy as np
import pandas as pd
import joblib

# Load model
consumption_model_path = "../../Model/Exports/Consumption Model/random_forest_consumption_model_location_lane_month.pkl"
consumption_model = joblib.load(consumption_model_path)


def predict_consumption(lat, lon, lane_count, month):
    query = np.array([[lat, lon, lane_count, month]])
    return float(consumption_model.predict(query)[0])

def predict_consumption_year(lat, lon, lane_count):
    monthly_consumption = []
    total = 0
    for i in range(1, 13):
        consumption = predict_consumption(lat, lon, lane_count, i)
        total += consumption
        monthly_consumption.append(consumption)
    return {"Total": total, "Monthly": monthly_consumption}

## Run

In [2]:
# Example run
predict_consumption(125.07623, 121.419053, 3, 4)

c:\Users\victo\anaconda3\envs\MachineLearning\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


1178.4033333333334

## Testing